In [12]:
import mido

mid = mido.MidiFile("StorkeTower.mid", clip=True)

for msg in mid:
    if msg.type == "program_change":
        print(msg)
    if msg.type == "set_tempo":
        print("Tempo set to", msg.tempo)
        bpm = round(6e7 / msg.tempo, 1)
        print("bpm:", bpm)

Tempo set to 320855
bpm: 187.0
channel 0 program 36
channel 9 program 0
channel 1 program 0
channel 2 program 32
channel 3 program 0
channel 9 program 0
channel 4 program 81
channel 5 program 80
channel 9 program 0
channel 6 program 81
channel 9 program 0
channel 7 program 42
channel 8 program 64
channel 10 program 64
channel 11 program 64
channel 12 program 64
channel 13 program 64
channel 14 program 64
channel 15 program 64


In [4]:
# Initialize lists to store MIDI data and output
midi_events = []
output = []

# Extract relevant MIDI events and convert them to dictionaries
for event in mid:
    if event.type in ["note_on", "note_off", "time_signature"]:
        midi_events.append(event.dict())

# Convert delta times to absolute times
current_time = 0
for event in midi_events:
    event["time"] += current_time
    current_time = event["time"]

    # Convert note_on events with 0 velocity to note_off events
    if event["type"] == "note_on" and event["velocity"] == 0:
        event["type"] = "note_off"

    # Prepare the event data for output
    event_data = []
    if event["type"] in ["note_on", "note_off"]:
        event_data = [event["type"], event["note"], event["time"], event["channel"]]
    elif event["type"] == "time_signature":
        event_data = [
            event["type"],
            event["numerator"],
            event["denominator"],
            event["time"],
        ]

    if event_data:
        output.append(event_data)

# Display the processed MIDI events
for event in output:
    print(event)

# Print the ticks per beat of the MIDI file
print(mid.ticks_per_beat)

['time_signature', 4, 4, 0]
['note_on', 60, 0, 0]
['note_off', 60, 1.22193798125, 0]
['note_on', 57, 1.224489, 0]
['note_off', 57, 2.44642698125, 0]
['note_on', 48, 2.4489780000000003, 0]
['note_off', 48, 4.8954049812500005, 0]
['note_on', 48, 4.897956000000001, 0]
['note_off', 48, 6.119893981250001, 0]
['note_on', 53, 6.122445000000001, 0]
['note_off', 53, 7.344382981250001, 0]
['note_on', 57, 7.346934000000001, 0]
['note_off', 57, 8.568871981250002, 0]
['note_on', 50, 8.571423000000001, 0]
['note_off', 50, 9.79336098125, 0]
['note_on', 57, 9.795912, 0]
['note_off', 57, 12.24233898125, 0]
['note_on', 59, 12.24489, 0]
['note_off', 59, 13.466827981249999, 0]
['note_on', 57, 13.469378999999998, 0]
['note_off', 57, 15.91580598125, 0]
['note_on', 60, 15.918356999999999, 0]
['note_off', 60, 17.140294981249998, 0]
['note_on', 52, 17.142846, 0]
['note_off', 52, 18.36478398125, 0]
['note_on', 55, 18.367335, 0]
['note_off', 55, 19.58927298125, 0]
['note_on', 53, 19.591824000000003, 0]
['note_of

In [5]:
# create a 31-element list of empty lists called "tracks"
tracks = [[] for _ in range(31)]

# put all notes in the right track
for i in output:
    tracks[i[3]].append(i)

# put all notes in the right order
for i in tracks:
    i.sort(key=lambda x: x[2])

# print all tracks
for i in tracks:
    print(i)

TypeError: list indices must be integers or slices, not float

In [6]:
import uuid

tracks_with_duration = [[] for _ in range(31)]

# calculate the duration of each note
for i in range(len(tracks)):
    for j in range(len(tracks[i])):
        if tracks[i][j][0] == "note_on":
            for k in range(j, len(tracks[i])):
                if tracks[i][k][0] == "note_off" and tracks[i][k][1] == tracks[i][j][1]:
                    tracks_with_duration[i].append(
                        {
                            "id": str(uuid.uuid4()),
                            "pitch": float(tracks[i][j][1]),
                            "time": tracks[i][j][2],
                            "duration": round((tracks[i][k][2] - tracks[i][j][2]) * bpm / 60, 4),
                        }
                    )
                    break

# print all tracks with duration
for i in tracks_with_duration:
    print(i)

[{'id': '77f51134-2a6b-40d8-817a-1c13a07fed21', 'pitch': 60.0, 'time': 0, 'duration': 1.2219}, {'id': 'a99232cb-e074-4a60-ad4b-b3788ac2b4ec', 'pitch': 57.0, 'time': 1.224489, 'duration': 1.2219}, {'id': 'a85a0286-778b-455d-8e4a-90ff5c74b6f8', 'pitch': 48.0, 'time': 2.4489780000000003, 'duration': 2.4464}, {'id': 'bc9f9bfc-645c-49d2-8882-f8b42b7ff933', 'pitch': 48.0, 'time': 4.897956000000001, 'duration': 1.2219}, {'id': 'd70b8388-43cb-431a-9f38-a4de74cdc802', 'pitch': 53.0, 'time': 6.122445000000001, 'duration': 1.2219}, {'id': '166ffc5d-3cb0-417b-813e-0fcf3c582450', 'pitch': 57.0, 'time': 7.346934000000001, 'duration': 1.2219}, {'id': 'a4ae5177-f7e1-4e6f-9e0d-ca2d5bf28e5d', 'pitch': 50.0, 'time': 8.571423000000001, 'duration': 1.2219}, {'id': 'bb4df94a-7d23-4422-8f27-e61d22c38892', 'pitch': 57.0, 'time': 9.795912, 'duration': 2.4464}, {'id': '71336adb-fc68-4748-8365-a8cddf755e52', 'pitch': 59.0, 'time': 12.24489, 'duration': 1.2219}, {'id': 'da90f69b-b762-4a96-be92-d86f664ae06c', 'pit

In [7]:
total_time = 0
for msg in mid:
    total_time += msg.time

print("Total time:", total_time)

size = round(total_time * bpm / 60 / 4)

print("Size:", size)

Total time: 58.163247000000005
Size: 15


In [8]:
# program_change channel=0 program=36 time=0
# program_change channel=1 program=0 time=0
# program_change channel=2 program=32 time=0
# program_change channel=3 program=0 time=0
# program_change channel=4 program=81 time=0
# program_change channel=5 program=80 time=0
# program_change channel=6 program=81 time=0
# program_change channel=7 program=42 time=0
# program_change channel=8 program=64 time=0
# program_change channel=9 program=0 time=0
# program_change channel=10 program=64 time=0
# program_change channel=11 program=64 time=0
# program_change channel=12 program=64 time=0
# program_change channel=13 program=64 time=0
# program_change channel=14 program=64 time=0
# program_change channel=15 program=64 time=0
channel_patches = [
    35,
    31,
    30,
    31,
    9,
    80,
    5,
    16,
    11,
    33,
    11,
    11,
    11,
    11,
    11,
    11,
    11,
    11,
    11,
    11,
    11,
    11,
    11,
    11,
    11,
    11,
    11,
    11,
    11,
    11,
    11
]

In [9]:
res = {
    "value0": {
        "bpm": bpm,
        "patterns": [
            {
                "length": size,
                "name": "",
                "id": str(uuid.uuid4()),
                "tracks": tracks_with_duration
            }
        ],
        "channels": [
            {
                "patch": channel_patches[i],
                "atk": 0.0010000000474974514,
                "rel": 0.12999999523162843,
                "volume": 1.0,
                "pan": 0.0,
                "lp": 20000.0,
                "tune": 0.0,
                "soft": True
            }
            for i in range(31)
        ]
    }
}

In [10]:
# output to song.json
import json

with open("song.json", "w") as f:
    json.dump(res, f, indent=4)
    